# 🏥 Deterministic Agent Workflow for Healthcare Journey Optimization

This notebook showcases a complete OpenAI Agent SDK pipeline for analyzing and improving healthcare journey efficiency using three orchestration patterns:

1. ✅ **Deterministic Workflow** — simple baseline flow with hardcoded patient data
2. ⚡ **Enhanced Flow** — batch analysis of synthetic patient records using `CodeInterpreterTool`, calling Zapier MCP server for composing and emailing final report

Each step includes **code, agent tools, and detailed markdown** explanations.


# Initial Setup

In [ ]:
from colabcode import ColabCode
ColabCode(
    port=10000)

In [1]:
# 📦 Install OpenAI Agents SDK and other dependencies
!pip install -q openai-agents openai tiktoken pandas matplotlib seaborn


    python-dotenv (>=0.13.*) ; extra == 'standard'
                   ~~~~~~~^
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.1.5 requires pyqt5<5.13, which is not installed.
spyder 5.1.5 requires pyqtwebengine<5.13, which is not installed.


## 🔐 Set your OpenAI API Key securely

In [3]:
# 🔐 Set your OpenAI API Key securely
from google.colab import userdata
import os

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

ModuleNotFoundError: No module named 'google'

## 📚 Import packages

In [ ]:
# 📚 Import packages
import pandas as pd
import numpy as np
from dataclasses import dataclass
from typing import List, Dict
from agents import Agent, Runner, function_tool, RunContextWrapper
from agents import CodeInterpreterTool
from agents.mcp import MCPServer, MCPServerSse
import json
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import random
from agents.mcp import MCPServer, MCPServerSse
from agents.model_settings import ModelSettings

# ✅ Part 1: Deterministic Workflow – Baseline Agent Flow with Hardcoded Input

## 🏥 Define structured input for patient journey using dataclass

In [ ]:
# 🏥 Define structured input for patient journey
@dataclass
class PatientJourney:
    admission_time: str
    discharge_time: str
    wait_times: dict  # e.g. {"triage": 40, "consultation": 75, "labs": 30}
    readmitted_within_30_days: bool

### Define function tools to analyze wait time and check readmission and provide the Patient Journey object as context to be available to the tools

In [ ]:
@function_tool
def analyze_wait_times(wrapper: RunContextWrapper[PatientJourney]) -> str:
    high_wait = {step: t for step, t in wrapper.context.wait_times.items() if t > 60}
    if high_wait:
        return f"Bottlenecks found in: {', '.join(high_wait.keys())}"
    return "No major wait time bottlenecks detected."


In [ ]:
@function_tool
def check_readmission(wrapper: RunContextWrapper[PatientJourney]) -> str:
    if wrapper.context.readmitted_within_30_days:
        return "🚨 Readmitted within 30 days"
    return "✅ No readmission"


### **Define wait_agent, readmit_agent and recommendation_agent and provide the PatientJourney object as context to the agents**

In [ ]:
wait_agent = Agent[PatientJourney](
    name="WaitTimeAgent",
    instructions="Analyze wait times and identify any delays over 60 minutes and use the patient journey information provided as {patient_journey_dict}",
    model="gpt-4o",
    tools=[analyze_wait_times],
    output_type=str
)

readmit_agent = Agent[PatientJourney](
    name="ReadmitAgent",
    instructions="Check if patient was readmitted",
    model="gpt-4o-mini",
    tools=[check_readmission],
    output_type=str
)

recommendation_agent = Agent(
    name="RecommendationAgent",
    instructions="Given messages about wait times and readmission, generate a final summary recommendation.",
    model="gpt-4o-mini",
    output_type=str
)

### **Define the sequence of agents for deterministic flow**

In [ ]:
# 🔁 Step 4: Deterministic Execution Flow
async def run_pipeline(journey: PatientJourney):
    print("🔍 Step 1: Wait time analysis...")
    # Convert the Pydantic model to a dictionary before passing to Runner.run

    wait_result = await Runner.run(wait_agent, input = "Use the use patient journey provided from {journey} ", context=journey)
    print("\t", wait_result.final_output)

    print("🔍 Step 2: Readmission check...")
    readmit_result = await Runner.run(readmit_agent, input = "Use the use patient journey provided from {journey} ", context=journey)
    print("\t", readmit_result.final_output)

    print("🧠 Step 3: Generating final recommendation...")
    # Pass the two string results as separate items in a list to Runner.run
    result3 = await Runner.run(recommendation_agent,input = "Please review triage and consultation times and evaluate causes of readmission ", context=[wait_result.final_output, readmit_result.final_output])
    print("\n✅ Recommendation:")
    print(result3.final_output)

### **Define the sample data**

In [ ]:
example_input = PatientJourney(
    admission_time="2025-06-10T10:00:00",
    discharge_time="2025-06-12T14:00:00",
    wait_times={"triage": 45, "consultation": 80, "labs": 25},
    readmitted_within_30_days=True
)

### **Execute deterministic flow by providing sample data**

In [ ]:
await run_pipeline(example_input)

🔍 Step 1: Wait time analysis...
	 The analysis identified a bottleneck in the consultation phase. This may be causing delays over 60 minutes in wait times. Could you provide more details or specific patient journey information for a deeper analysis?
🔍 Step 2: Readmission check...
	 The patient was readmitted within 30 days. If you need further details or next steps, let me know!
🧠 Step 3: Generating final recommendation...

✅ Recommendation:
Certainly! Here’s a summary recommendation based on the evaluation of triage and consultation times, along with considerations of readmission causes:

### Summary Recommendation

1. **Triage and Consultation Times**:
   - **Assessment**: Review current protocols for triage to ensure timely assessment of patients. Track wait times to identify any bottlenecks.
   - **Improvement**: Implement a triage team dedicated to reducing wait times during peak hours. Consider utilizing telemedicine for initial consultations to expedite the process.

2. **Causes

#⚡ Part 2: Enhanced Workflow – Analyze Trends with CodeInterpreterTool

We now scale the solution to handle multiple patient journeys by generating synthetic data and analyzing it using OpenAI’s **CodeInterpreterTool**.

### Goals:
- Generate 100 synthetic records and save as CSV
- Load the data and compute:
  - Average wait time per service
  - Services with most delays
  - Readmission rate
- Show visualizations


In [ ]:
# 📊 Generate synthetic patient data

def generate_synthetic_journeys(n=100):
    data = []
    for _ in range(n):
        admit = datetime(2025, 6, 1) + timedelta(days=random.randint(0, 14))
        discharge = admit + timedelta(days=random.randint(1, 3))
        wait_times = {
            "triage": random.randint(10, 90),
            "consultation": random.randint(20, 120),
            "labs": random.randint(15, 60)
        }
        readmit = random.choice([True] * 3 + [False] * 7)
        data.append({
            "admission_time": admit.isoformat(),
            "discharge_time": discharge.isoformat(),
            "wait_times": json.dumps(wait_times),
            "readmitted_within_30_days": readmit
        })
    return pd.DataFrame(data)

df = generate_synthetic_journeys()
df.to_csv("synthetic_patient_data.csv", index=False) # Changed to .csv
df.head()

,admission_time,discharge_time,wait_times,readmitted_within_30_days
0,2025-06-04T00:00:00,2025-06-05T00:00:00,"{""triage"": 49, ""consultation"": 87, ""labs"": 44}",False
1,2025-06-13T00:00:00,2025-06-15T00:00:00,"{""triage"": 73, ""consultation"": 108, ""labs"": 28}",False
2,2025-06-08T00:00:00,2025-06-09T00:00:00,"{""triage"": 23, ""consultation"": 58, ""labs"": 31}",True
3,2025-06-12T00:00:00,2025-06-15T00:00:00,"{""triage"": 26, ""consultation"": 49, ""labs"": 51}",True
4,2025-06-09T00:00:00,2025-06-12T00:00:00,"{""triage"": 50, ""consultation"": 95, ""labs"": 47}",False


## Define the trend agent to take the data and use code interpreter

In [ ]:
from pydantic import BaseModel
from typing import Dict, List

@dataclass
class TrendAnalysisResult():
    avg_wait_times: Dict[str, float]
    delays_over_60min: Dict[str, int]
    readmission_rate_percent: float
    summary_text: str  # Optional: summary paragraph or markdown text


In [ ]:
from agents import CodeInterpreterTool,AgentOutputSchema
# 🤖 Use CodeInterpreterTool to analyze CSV trends


trend_agent = Agent[TrendAnalysisResult](
    name="TrendAnalyzer",
    instructions="""Analyze the CSV file. Return:
1. Average wait times per service
2. Services most frequently delayed (>60 min)
3. Readmission rate as percentage
Also generate a bar chart for average wait time.in the format {TrendAnalysisResult}""" ,
    model="gpt-4o-mini",
    tools=[
            CodeInterpreterTool(
                tool_config={"type": "code_interpreter", "container": {"type": "auto"}},
            )
        ],
    output_type=AgentOutputSchema(TrendAnalysisResult,strict_json_schema=False)
)



## 📤 Email Report via MCPTool and Zapier Gmail

This final part demonstrates the **usage of MCP**:
- A report agent is called dynamically to generate a system summary
- The output is sent as an email using `MCPServerSSE` (which calls a Zapier Gmail webhook)


### Synthesize report function to format the report

In [ ]:
@function_tool(strict_mode=False)
def synthesize_report(wrapper: RunContextWrapper[TrendAnalysisResult]) -> str:
    result = wrapper.context  # This is now a TrendAnalysisResult Pydantic object

    # 🧾 Format wait time averages
    avg_wait_lines = "\n".join(f"- {service.title()}: {minutes:.2f} min"
                               for service, minutes in result.avg_wait_times.items())

    # 🛑 Format delay frequencies
    delay_lines = "\n".join(f"- {service.title()}: {count} occurrences"
                            for service, count in result.delays_over_60min.items())

    # 📧 Compose report text
    return f"""📋 Healthcare System Summary Report

🕒 Average Wait Times:
----------------------
{avg_wait_lines}

⚠️ Delays Over 60 Minutes:
--------------------------
{delay_lines}

🔁 Readmission Rate:
---------------------
{result.readmission_rate_percent:.1f}%

📝 Summary:
-----------
{result.summary_text}

Regards,
AI Monitoring System
"""


### MCP implementation

In [ ]:
# Convert file to text for code interpreter
csv_text = open("synthetic_patient_data.csv", "r").read()
url = userdata.get('MCP_SERVER_URL_SSE')
# Step 2: Manually initialize the MCPServerSse
mcp_server = MCPServerSse(
    name="MCP_Email",
    params={
        "url": url
    },
    client_session_timeout_seconds = 30

)


# Step 3 Define the agent that uses the MCP server
report_agent = Agent(
    name="EmailAgent",
    instructions="Generate a professional email summary by calling synthesize_report tool , based on wait and trend analysis recieved from the trend_agent and reciepient name is Anjali Jain , It should not include the visuals in emails.",
    mcp_servers=[mcp_server],
    model_settings=ModelSettings(tool_choice="required"),
    model="gpt-4o-mini",
    tools=[synthesize_report],
    output_type=str,
)

# Step 4: Generate the report
analysis_result = await Runner.run(trend_agent, input=csv_text)
print(analysis_result)
# Step 5: Await entry (equivalent to `async with`)
await mcp_server.__aenter__()
# Step 6: Run the agent with the analysis result
#result = await Runner.run(report_agent, input=analysis_result.final_output)
result = await Runner.run(report_agent, input = "generate the report by calling synthesize_report tool first and then send the synthesized data to create the email",context=analysis_result.final_output)
print("📧 Email result:", result.final_output)

# Step 7: Clean up the MCP server session
await mcp_server.__aexit__(None, None, None)


RunResult:
- Last agent: Agent(name="TrendAnalyzer", ...)
- Final output (TrendAnalysisResult):
    TrendAnalysisResult(avg_wait_times={'triage': 45.15, 'consultation': 73.81, 'labs': 38.22}, delays_over_60min={'triage': 27, 'consultation': 66}, readmission_rate_percent=30.0, summary_text='The average wait times are 45.15 minutes for triage, 73.81 minutes for consultation, and 38.22 minutes for labs. There are 27 delays over 60 minutes in triage and 66 in consultation. The readmission rate within 30 days is 30%.')
- 5 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)
📧 Email result: The email summary has been successfully sent to Anjali Jain. Here are the details of the email:

### Subject: Healthcare System Summary Report

**Body:**
```
Dear Anjali Jain,

Please find below the summary of our healthcare system report regarding wait times and additional metrics:

🕒 Average Wait Times:
----------------------
- 